![image](headerBI.png)

# Proyecto 1 

Integrantes:
- Federico Melo Barrero, 202021525, f.melo. Sección 1.
- Shadith Pérez Rivera, 202014687, s.perezr. Sección 2.

## Enfoque Analítico

## Instalación e importación de librerías.

In [1]:
# Instalación de librerias
import pandas as pd
import ydata_profiling as pp
import numpy as np
import sys
import nltk
import random
from num2words import num2words
from collections import Counter 

import re, string, unicodedata
import inflect
import os
import seaborn as sns
import matplotlib.pyplot as plt
from langdetect import detect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.stem import SnowballStemmer
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.base import BaseEstimator, ClassifierMixin

#redNeuronal
import gensim
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# model = gensim.models.KeyedVectors.load_word2vec_format('/work/wiki.es.vec', binary=False)
from keras.utils import to_categorical
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


2023-10-15 17:27:20.124277: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 17:27:20.368566: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-15 17:27:20.368609: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-15 17:27:20.437383: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-15 17:27:22.429368: W tensorflow/stream_executor/platform/de

True

## 1. Carga de los datos

In [2]:
# Uso de la libreria pandas para la lectura de archivos
dataO = pd.read_excel('cat_6716.xls')

## 2. Entendimiento de los datos

El proposito es obtener información general sobre el conjunto de datos para comprender su estructura y características. Para empezar crearemos una copia de los datos para no modificar el orginal. 

In [3]:
data = dataO.copy()

In [4]:
rows, cols = data.shape
print(f"Se parte de un dataset con {rows} filas y {cols} columnas")

Se parte de un dataset con 3000 filas y 2 columnas


In [5]:
print("Se visualiza una muestra de los datos: ")
data.sample(5)

Se visualiza una muestra de los datos: 


,Textos_espanol,sdg
2158,Los grupos extremistas nacionales de EE. UU. h...,16
2541,Dos de los asuntos de derechos humanos más not...,16
1329,"Por el contrario, la participación de las ener...",7
1369,Promover iniciativas y alentar la inversión en...,7
1041,Se presentan resultados alternativos para un e...,7


### 2.1 Perfilamiento de los datos

In [6]:
rows, cols = data.shape
print(f"Se parte de un dataset con {rows} filas y {cols} columnas")

Se parte de un dataset con 3000 filas y 2 columnas


In [7]:
print("Se visualiza una muestra de los datos: ")
data.sample(5)

Se visualiza una muestra de los datos: 


,Textos_espanol,sdg
1206,"Esto preocupa a los desarrolladores, ya que el...",7
2270,Una etnografía de los servicios jurídicos pres...,16
238,El principal objetivo de la TNMN es proporcion...,6
2655,Este artículo explora la política internaciona...,16
408,Los precios del suministro de agua han sido má...,6


In [8]:
print("A continuación, se identifican los tipos de datos de cada columna, al igual que las filas vacías: ")
data.info()

A continuación, se identifican los tipos de datos de cada columna, al igual que las filas vacías: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Textos_espanol  3000 non-null   object
 1   sdg             3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [9]:

numeric_cols_count: int = len(data.select_dtypes(include=['int64', 'float64']).columns)
categorical_cols_count: int = len(data.select_dtypes(include=['object']).columns)

print(f"El dataset contiene {numeric_cols_count} variables numéricas y {categorical_cols_count} variables categóricas")

El dataset contiene 1 variables numéricas y 1 variables categóricas


Para comprender mejor cada dato, que representa una descripción de alguna problemática relacionada con los ODS de la ONU en algún entorno particular, se añaden columnas con:

- El número de palabras del texto.

- La palabra más frecuente en el texto.

- El palabra con mayor longitud del texto.

- La palabra con menor longitud que se encuentra en el texto.

In [10]:
textos = data.copy()
textos['Conteo'] = [len(x) for x in textos['Textos_espanol']]
textos['Moda'] = [Counter(x.split(" ")).most_common(1)[0][0] for x in textos['Textos_espanol']]
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['Textos_espanol']]
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['Textos_espanol']]

# Se realiza un perfilamiento de los datos con la librería pandas profiling
pp.ProfileReport(textos)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

En el reporte generado por ydata profiling se evidencia lo siguiente sobre el conjunto de registros:

- Todos los registros son distintos, de forma que cada dato es único.

- No hay datos faltantes (e.g. un texto sin clasificación).

- Hay exactamente tres categorías: 6, 7 y 16, ningún texto está categorizado con algo diferente.

- Las categorías están distribuidas de manera uniforme, hay exactamente 1000 datos de cada categoría.

- Los textos tienen en promedio 770 caracteres, el que menos tiene cuenta con 143 y el que más cuenta con 1616

- La gran mayoría de los textos tienen la palabra "de" como la más común. En casi todos los textos, la palabra más común es una palabra de parada.

- El promedio de tamaño para la palabra más larga de cada texto es de 16 caracteres, siendo la menor de 10 y la mayor de 109 caracteres (probablemente por una falta de espacio por un error).

- El promedio de tamaño para la palabra más corta de cada texto es de 1 caracter, probablemente la palabra "y" (que es una palabra de parada). 

Por último, se evidencia una leve correlación entre el número de caracteres de una descripción y la clasificación de la descripción en uno de los 3 posibles ODS: 6, 7 o 16. Esta correlación es menor al 50% y por ende no debería afectar en gran medida el modelo.

### 2.2 Análisis y calidad de los datos

Revisamos las dimensiones de calidad: unicidad, completitud, consistencia, validez.

### 2.2.1 Unicidad 

Se contempla que cada registro sea único, que no existan duplicados, para así garantizar que cada dato tiene el mismo peso en el entrenamiento del modelo.

Como mencionado anteriormente, en el análisis de datos no se encontró ningún dato duplicado, por lo cual no hay necesidad de eliminar ningún registro.

In [11]:
data.duplicated().sum()

0

### 2.2.2 Completitud

Se revisa la completitud de los datos en cada una de las columnas, para cerciorarse de que cada texto corresponde a una clasificación como O y que cada clasificación está asociada a una descripción.

Se encontró que efectivamente los datos están completos, no hay ninguna fila con alguna entrada faltante, todos los textos cuentan con una clasificación y no no hay ninguna clasificación a la cual no le corresponda un texto.

In [12]:
completitud = data.count() / len(data) * 100
print(completitud)

Textos_espanol    100.0
sdg               100.0
dtype: float64


### 2.2.4 Validez

Se contempla que todos los registros sean válidos, es decir, en este caso, que cada una de las descripciones efectivamente sea un dato de texto con un mínimo de caracteres y que cada una de las clasificaciones efectivamente sea un número en el conjunto {6, 7, 16}.

In [13]:
validez_texto = data["Textos_espanol"].apply(lambda x: isinstance(x, str) and len(x) >= 10)
porcentaje_invalidos = (1 - validez_texto.mean()) * 100
print(f"El {porcentaje_invalidos:.2f}% de los datos de texto no son válidos")

El 0.00% de los datos de texto no son válidos


In [14]:
validez_categorias = data["sdg"].apply(lambda x: x in [6, 7, 16])
porcentaje_invalidos = (1 - validez_categorias.mean()) * 100
print(f"El {porcentaje_invalidos:.2f}% de las clasificaciones no son válidas")

El 0.00% de las clasificaciones no son válidas


### 2.2.3 Consistencia

Por último, después de los anterior, se evidencia que los datos son consistentes al haber consistencia estructural (pues los esos datos de texto están almacenados como cadenas de caracteres y las clasificaciones, que son datos numéricos, están almacenadas bajo tipos numéricos), semántica (los datos que representan el mismo concepto tienen la misma reprresentación, por ejemplo no hay dos clasificaciones "6" y "seis", únicamente hay 6) y de contenido (pues los textos no se contradicen directamente entre sí).

## 3. Limpieza y preparacion de los datos

In [15]:
column_names = data.columns.tolist()
print(column_names)


['Textos_espanol', 'sdg']


In [16]:
# Creamos una nueva columna para almacenar el idioma detectado
data['idioma'] = data['Textos_espanol'].apply(detect)

# Calculamos la proporción de cada idioma
idiomas = data['idioma'].value_counts(normalize=True)
print(idiomas)


es    0.998667
en    0.001333
Name: idioma, dtype: float64


In [17]:
#eliminar los que estan en ingles
data = data[data['idioma'] != 'en']

In [18]:
#Ver si esta balanceado
clase_counts = data['sdg'].value_counts()
print(clase_counts)


6     1000
16    1000
7      996
Name: sdg, dtype: int64


In [19]:
# Calcular las frecuencias de las clases
clase_counts = data['sdg'].value_counts()

# Gráfico de barras
plt.figure(figsize=(8, 6))
sns.barplot(x=clase_counts.index, y=clase_counts.values)

# Personalizar el gráfico
plt.title('Distribución de ODS')
plt.xlabel('ODS')
plt.ylabel('Frecuencia')

# Mostrar el gráfico
plt.show()


In [20]:
def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        new_words.append(word.lower())
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = num2words(word, lang='es')
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    stop_words = set(stopwords.words('spanish'))
    new_words = [word for word in words if word not in stop_words]
    return new_words

def preprocessing(text):
    words = text.split()  # Tokeniza el texto en palabras
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return ' '.join(words)  # Une las palabras procesadas en un texto

data['Textos_espanol'] = data['Textos_espanol'].apply(preprocessing)

In [21]:
# muestra aleatoria de ejemplos
sample_indices = random.sample(range(len(data)), 3)  
sample_data = dataO.iloc[sample_indices]

#función de preprocesamiento a los ejemplos seleccionados
sample_data['Textos_preprocesados'] = sample_data['Textos_espanol'].apply(preprocessing)

#ejemplos antes y después del preprocesamiento
for index, row in sample_data.iterrows():
    print("Original:")
    print(row['Textos_espanol'])
    print("\nPreprocesado:")
    print(row['Textos_preprocesados'])
    print("\n---------------------------------\n")

Original:
En países con altas tasas de interés y/o monedas inestables, la necesidad de un mecanismo tipo ESCO es mayor, pero los desafíos involucrados en el establecimiento de una ESCO también serán particularmente significativos. Los empresarios ecológicos a pequeña escala pueden ser parte del entorno propicio a nivel local. Estos empresarios pueden contribuir a mejorar el desempeño ambiental en el sector energético al ofrecer bienes y servicios ambientalmente beneficiosos. El proyecto proporcionó equipos, materiales y apoyo empresarial para ayudar a los emprendedores a establecer un modelo empresarial sostenible (Carbon Positive, 2010). Existen otras oportunidades comerciales a pequeña escala en áreas tales como la producción y venta de pellets de combustible de biomasa de desecho, la producción de plántulas en viveros comunitarios a pequeña escala, la fabricación de otros tipos de equipos (por ejemplo, cocinas solares) y más.

Preprocesado:
paises altas tasas interes monedas inestab

## Tokenización

In [22]:
# Tokeniza los textos y aplicar el preprocesamiento
data['words'] = data['Textos_espanol'].apply(word_tokenize)

for i in range(1):
    original_text = data['Textos_espanol'][i]
    tokenized_text = word_tokenize(original_text)
    preprocessed_text = preprocessing(original_text)
    
print("Original:")
print(original_text)
print("\nTexto tokenizado:")
print(tokenized_text)
print("\nTexto preprocesado:")
print(preprocessed_text)
print("\n---------------------------------\n")

Original:
importante destacar ano sequia espera disminuyan aportes aumente demanda agua cultivos vegetacion provocara reduccion caudal rios estan conectados hidrologicamente acuifero asi menor disponibilidad agua ecosistemas dependientes aguas subterraneas externalidades espaciales pueden incluir agotamiento cursos agua descenso nivel freatico local ejemplo bombeo continuo pozo b provocara agotamiento cursos agua reduccion caudal

Texto tokenizado:
['importante', 'destacar', 'ano', 'sequia', 'espera', 'disminuyan', 'aportes', 'aumente', 'demanda', 'agua', 'cultivos', 'vegetacion', 'provocara', 'reduccion', 'caudal', 'rios', 'estan', 'conectados', 'hidrologicamente', 'acuifero', 'asi', 'menor', 'disponibilidad', 'agua', 'ecosistemas', 'dependientes', 'aguas', 'subterraneas', 'externalidades', 'espaciales', 'pueden', 'incluir', 'agotamiento', 'cursos', 'agua', 'descenso', 'nivel', 'freatico', 'local', 'ejemplo', 'bombeo', 'continuo', 'pozo', 'b', 'provocara', 'agotamiento', 'cursos', 'ag

## Normalización

In [23]:
def stem_words(words):
    stemmer = SnowballStemmer('spanish')
    return [stemmer.stem(word) for word in words]

data['tokens'] = data['words'].apply(stem_words)

#muestra aleatoria de ejemplos
sample_indices = random.sample(range(len(data)), 5)  # Cambia 5 por el número de ejemplos que desees mostrar
sample_data = data.iloc[sample_indices]

#función de stemming a los ejemplos seleccionados
sample_data['stemmed_tokens'] = sample_data['words'].apply(stem_words)

#ejemplos antes y después del stemming
for index, row in sample_data.iterrows():
    print("Original:")
    print(row['words'])
    print("\nDespués del stemming:")
    print(row['stemmed_tokens'])
    print("\n---------------------------------\n")

Original:
['incentivar', 'uso', 'gas', 'natural', 'consumo', 'sido', 'exento', 'impuestos', 'exenciones', 'parciales', 'totales', 'tambien', 'aplican', 'combustibles', 'incluidos', 'utilizados', 'generacion', 'electricidad', 'propulsion', 'barcos', 'incluidos', 'barcos', 'pesca', 'agricultura', 'instalaciones', 'participan', 'acuerdos', 'negociados', 'ahorro', 'energia', 'seccion', '41', 'eu', 'ets', 'asi', 'biocombustibles', 'seccion', '52', 'aplica', 'tasa', 'mas', 'baja', 'impuesto', 'valor', 'agregado', 'iva', 'combustible', 'calefaccion', 'consumo', 'domestico', 'gas', 'natural', 'electricidad']

Después del stemming:
['incentiv', 'uso', 'gas', 'natural', 'consum', 'sid', 'exent', 'impuest', 'exencion', 'parcial', 'total', 'tambi', 'aplic', 'combust', 'inclu', 'utiliz', 'generacion', 'electr', 'propulsion', 'barc', 'inclu', 'barc', 'pesc', 'agricultur', 'instal', 'particip', 'acuerd', 'negoci', 'ahorr', 'energi', 'seccion', '41', 'eu', 'ets', 'asi', 'biocombust', 'seccion', '52', 

# Modelos

## Red Neuronal

In [24]:
X = data['Textos_espanol']
y = data['sdg']  
X_train,X_test,y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train,X_val,y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

tfidf_vectorizer = TfidfVectorizer()  # Limita el número de características si es necesario
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()
X_val_tfidf = tfidf_vectorizer.transform(X_val).toarray()


X_train_tfidf = np.array(X_train_tfidf, dtype=np.float32)
X_test_tfidf= np.array(X_test_tfidf, dtype=np.float32)
X_val_tfidf= np.array(X_val_tfidf, dtype=np.float32)

In [26]:
X_train_tfidf[2]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [27]:
model = Sequential()
model.add(Dense(64, input_shape=(X_train_tfidf.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # num_classes es el número de categorías

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-10-15 17:28:54.549725: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-15 17:28:54.549784: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-15 17:28:54.549859: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-54529ab6-cd7d-40b9-9312-e83cf4ce27d0): /proc/driver/nvidia/version does not exist
2023-10-15 17:28:54.550313: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
from keras.utils import to_categorical
from tensorflow import keras

label_mapping = {6: 0, 7: 1, 16: 2}

# Mapear las etiquetas originales a índices
y_train = [label_mapping[label] for label in y_train]
y_test = [label_mapping[label] for label in y_test]
y_val = [label_mapping[label] for label in y_val]

# # Convierte las etiquetas a formato one-hot encoding
y_train = to_categorical(y_train, num_classes=3)  # 3 categorías en este ejemplo
y_test = to_categorical(y_test, num_classes=3)
y_val = to_categorical(y_val, num_classes=3)


In [29]:
my_callback = [
    keras.callbacks.EarlyStopping(patience = 5), # Dejar de entrenar si no mejora
    keras.callbacks.ModelCheckpoint(filepath = 'best_model.h5', save_best_only = True), # Guardar el mejor modelo
    keras.callbacks.TensorBoard(log_dir = 'my_log_dir', histogram_freq = 1) # Guardar los logs para tensorboard
]

model.fit(X_train_tfidf, y_train, epochs=20, batch_size=128, validation_data=(X_val_tfidf, y_val), callbacks=my_callback)


Epoch 1/20
15/15 [==============================] - 3s 140ms/step - loss: 1.0633 - accuracy: 0.6670 - val_loss: 0.9942 - val_accuracy: 0.9062
Epoch 2/20
15/15 [==============================] - 1s 91ms/step - loss: 0.9142 - accuracy: 0.9327 - val_loss: 0.8434 - val_accuracy: 0.9229
Epoch 3/20
15/15 [==============================] - 2s 106ms/step - loss: 0.7426 - accuracy: 0.9687 - val_loss: 0.6950 - val_accuracy: 0.9479
Epoch 4/20
15/15 [==============================] - 1s 89ms/step - loss: 0.5823 - accuracy: 0.9812 - val_loss: 0.5604 - val_accuracy: 0.9688
Epoch 5/20
15/15 [==============================] - 1s 93ms/step - loss: 0.4456 - accuracy: 0.9901 - val_loss: 0.4503 - val_accuracy: 0.9729
Epoch 6/20
15/15 [==============================] - 1s 88ms/step - loss: 0.3391 - accuracy: 0.9937 - val_loss: 0.3658 - val_accuracy: 0.9771
Epoch 7/20
15/15 [==============================] - 1s 94ms/step - loss: 0.2584 - accuracy: 0.9948 - val_loss: 0.3024 - val_accuracy: 0.9771
Epoch 8/20


In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1075200   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 1,075,395
Trainable params: 1,075,395
Non-trainable params: 0
_________________________________________________________________


In [31]:
from tensorflow.keras.models import load_model

# Load the model from a file
model = load_model('/work/best_model.h5')

# You can now use the loaded model for predictions or further training

y_pred_prob= model.predict(X_test_tfidf)
y_pred = np.argmax(y_pred_prob, axis=1)
y_test = np.argmax(y_test, axis=1)

19/19 [==============================] - 0s 7ms/step


In [32]:
# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Calcular la precisión por clase
precision = precision_score(y_test, y_pred, average='macro')  # 'macro' calcula la precisión promedio
print(f'Precisión (Macro): {precision:.2f}')

# Calcular la recuperación (recall) por clase
recall = recall_score(y_test, y_pred, average='macro')  # 'macro' calcula la recuperación promedio
print(f'Recuperación (Macro): {recall:.2f}')

# Calcular la puntuación F1 por clase
f1 = f1_score(y_test, y_pred, average='macro')  # 'macro' calcula la puntuación F1 promedio
print(f'Puntuación F1 (Macro): {f1:.2f}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(conf_matrix)

# Imprimir el informe de clasificación
class_report = classification_report(y_test, y_pred)
print('Informe de Clasificación:')
print(class_report)

Accuracy: 0.97
Precisión (Macro): 0.97
Recuperación (Macro): 0.97
Puntuación F1 (Macro): 0.97
Matriz de Confusión:
[[201   9   3]
 [  4 195   1]
 [  0   0 187]]
Informe de Clasificación:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       213
           1       0.96      0.97      0.97       200
           2       0.98      1.00      0.99       187

    accuracy                           0.97       600
   macro avg       0.97      0.97      0.97       600
weighted avg       0.97      0.97      0.97       600



## TF_IDF

In [33]:
# Crear un objeto TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Aplicar TF-IDF a los tokens
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Textos_espanol'])

# Obtener la representación TF-IDF en un DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenar el DataFrame de TF-IDF con tu DataFrame original
data = pd.concat([data, tfidf_df], axis=1)

# Mostrar las primeras filas del DataFrame resultante
data.head()

,Textos_espanol,sdg,idioma,words,tokens,00000002,00000004,00015,001,002,...,zn,zona,zonas,zones,zonificacia3n,zonificacion,zout30,zre,zuo,zusammenarbeit
0,importante destacar ano sequia espera disminuy...,6.0,es,"[importante, destacar, ano, sequia, espera, di...","[import, destac, ano, sequi, esper, disminu, a...",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,gran cantidad literatura australia area sugier...,6.0,es,"[gran, cantidad, literatura, australia, area, ...","[gran, cantid, literatur, australi, are, sugie...",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,procesos descentralizacion emprendidos serie a...,6.0,es,"[procesos, descentralizacion, emprendidos, ser...","[proces, descentralizacion, emprend, seri, act...",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,puede tener consecuencias sustanciales calidad...,6.0,es,"[puede, tener, consecuencias, sustanciales, ca...","[pued, ten, consecuent, sustancial, calid, agu...",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,funcion beneficio tambien incorpora parametros...,6.0,es,"[funcion, beneficio, tambien, incorpora, param...","[funcion, benefici, tambi, incorpor, parametr,...",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Regresion Logistica

In [34]:
data = data.dropna(subset=['sdg'])


In [35]:
spanish_stop_words = set(stopwords.words('spanish'))

X = data['Textos_espanol']
y = data['sdg']  

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [36]:
lr_pipeline_tf_idf = Pipeline([
    ('vectorize', TfidfVectorizer(stop_words=spanish_stop_words, lowercase=True)),
    ('model', LogisticRegression())
])

In [37]:
X_train

2213    articulo analiza procesos transferencia difusi...
271     garantizar arreglos gobernanza ayuden moviliza...
2906    capitulo examina evolucion hacia intervencion ...
369     utiliza sistema bloques decrecientes cobrar ex...
2889    siguiente introduccion revisa lagunas existent...
                              ...                        
763     envejecimiento infraestructuras falta capacida...
835     condujo grave problema energetico llevo kirgui...
1655    existe clara compensacion ahorro energia rendi...
2611    articulo pone relieve reto desigualdad medioam...
2736    mal administrativo siniestro acecha sombras ba...
Name: Textos_espanol, Length: 2396, dtype: object

In [38]:
model_name = 'lr_pipeline_tf_idf'

lr_pipeline_tf_idf.fit(X_train, y_train)


# Ahora puedes utilizar lr_pipeline_tf_idf para hacer predicciones en tus datos de prueba
y_pred = lr_pipeline_tf_idf.predict(X_test)

In [39]:
y_pred

array([ 7.,  6.,  7., 16.,  6.,  7.,  6., 16.,  7.,  6., 16., 16.,  7.,
        6.,  6., 16.,  6.,  6., 16., 16.,  7., 16., 16., 16.,  6., 16.,
        6., 16.,  7.,  6.,  6.,  7.,  7.,  7., 16.,  7.,  6.,  6.,  7.,
        7., 16.,  7.,  6.,  6.,  6.,  6.,  6.,  7., 16.,  7.,  6., 16.,
        6., 16.,  6., 16.,  7., 16., 16.,  7., 16.,  7., 16.,  7.,  6.,
        6., 16.,  6.,  7., 16.,  6.,  6.,  7., 16.,  7.,  7.,  7., 16.,
        6., 16., 16.,  6., 16., 16.,  6.,  7.,  6.,  7.,  7., 16.,  7.,
        6.,  7.,  7.,  6., 16.,  6.,  6.,  6.,  6., 16., 16., 16.,  7.,
        6.,  7.,  7.,  7., 16., 16., 16.,  7.,  7.,  7., 16.,  6., 16.,
       16.,  6., 16.,  6.,  6.,  7.,  7., 16., 16., 16., 16., 16.,  6.,
        7., 16.,  7.,  6.,  6.,  6., 16.,  7.,  7.,  7.,  7.,  6., 16.,
       16.,  7.,  6., 16., 16.,  6.,  7.,  6.,  6.,  6.,  6., 16.,  6.,
       16.,  7.,  7., 16., 16.,  6.,  6.,  7.,  6., 16., 16., 16.,  6.,
       16.,  7.,  7.,  7., 16., 16.,  7.,  7.,  7.,  7.,  6.,  7

In [40]:
 
# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Calcular la precisión por clase
precision = precision_score(y_test, y_pred, average='macro')  # 'macro' calcula la precisión promedio
print(f'Precisión (Macro): {precision:.2f}')

# Calcular la recuperación (recall) por clase
recall = recall_score(y_test, y_pred, average='macro')  # 'macro' calcula la recuperación promedio
print(f'Recuperación (Macro): {recall:.2f}')

# Calcular la puntuación F1 por clase
f1 = f1_score(y_test, y_pred, average='macro')  # 'macro' calcula la puntuación F1 promedio
print(f'Puntuación F1 (Macro): {f1:.2f}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')

print(conf_matrix)

# Imprimir el informe de clasificación
class_report = classification_report(y_test, y_pred)
print('Informe de Clasificación:')
print(class_report)

Accuracy: 0.98
Precisión (Macro): 0.98
Recuperación (Macro): 0.98
Puntuación F1 (Macro): 0.98
Matriz de Confusión:
[[206   3   4]
 [  2 196   2]
 [  0   0 187]]
Informe de Clasificación:
              precision    recall  f1-score   support

         6.0       0.99      0.97      0.98       213
         7.0       0.98      0.98      0.98       200
        16.0       0.97      1.00      0.98       187

    accuracy                           0.98       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.98      0.98       600



## Naive Bayes

In [41]:
# Convertir los textos en una bolsa de palabras (CountVectorizer)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Textos_espanol'])
y = data['sdg']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Definir el modelo de Naive Bayes (MultinomialNB)
bys = MultinomialNB()

# Definir las particiones para la validación cruzada (por ejemplo, 5 particiones)
particiones = KFold(n_splits=10, shuffle=True, random_state=0)

# Definir los posibles valores de hiperparámetros a probar
param_grid = {'alpha': [0.1] + [x/2 for x in range(1, 21)]} # [0.1, 0.5, 1.0, 1.5, ..., 10.0]

# Utilizar GridSearchCV para encontrar los mejores hiperparámetros
bys = GridSearchCV(bys, param_grid, cv=particiones, n_jobs=-1)
bys.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y la precisión del modelo
best_params = bys.best_params_
accuracy = bys.best_score_
print("Best params: ", best_params)
print("Score: ", accuracy)



Best params:  {'alpha': 2.0}
Score:  0.9862168758716876


In [42]:
# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Calcular la precisión por clase
precision = precision_score(y_test, y_pred, average='macro')  # 'macro' calcula la precisión promedio
print(f'Precisión (Macro): {precision:.2f}')

# Calcular la recuperación (recall) por clase
recall = recall_score(y_test, y_pred, average='macro')  # 'macro' calcula la recuperación promedio
print(f'Recuperación (Macro): {recall:.2f}')

# Calcular la puntuación F1 por clase
f1 = f1_score(y_test, y_pred, average='macro')  # 'macro' calcula la puntuación F1 promedio
print(f'Puntuación F1 (Macro): {f1:.2f}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(conf_matrix)

# Imprimir el informe de clasificación
class_report = classification_report(y_test, y_pred)
print('Informe de Clasificación:')
print(class_report)

Accuracy: 0.98
Precisión (Macro): 0.98
Recuperación (Macro): 0.98
Puntuación F1 (Macro): 0.98
Matriz de Confusión:
[[206   3   4]
 [  2 196   2]
 [  0   0 187]]
Informe de Clasificación:
              precision    recall  f1-score   support

         6.0       0.99      0.97      0.98       213
         7.0       0.98      0.98      0.98       200
        16.0       0.97      1.00      0.98       187

    accuracy                           0.98       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.98      0.98       600



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=54529ab6-cd7d-40b9-9312-e83cf4ce27d0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>